<a href="https://colab.research.google.com/github/chong-z/NLG-project/blob/master/CS269_NLG_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS269 NLG Project: Generating Semi-Restricted Natural Language Adversarial Examples


## Setup

In [ ]:
!pip install transformers==3.0.2 pytorch-pretrained-bert==0.6.2 nlp torch nltk numpy tensorboardX pandas

!git clone https://github.com/chong-z/NLG-project.git
%cd NLG-project
!sh dowloaddata.sh

## Explore SST-2

In [19]:
import nlp
import pandas as pd
pd.options.display.min_rows = 20
pd.options.display.max_colwidth = 200

sst2_data = nlp.load_dataset('glue', 'sst2')['train']
df = pd.DataFrame(sst2_data)
display(df)

,idx,label,sentence
0,0,0,hide new secretions from the parental units
1,1,0,"contains no wit , only labored gags"
2,2,1,that loves its characters and communicates something rather beautiful about human nature
3,3,0,remains utterly satisfied to remain the same throughout
4,4,0,on the worst revenge-of-the-nerds clichés the filmmakers could dredge up
5,5,0,that 's far too tragic to merit such superficial treatment
6,6,1,"demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop ."
7,7,1,of saucy
8,8,0,a depressed fifteen-year-old 's suicidal poetry
9,9,1,are more deeply thought through than in most ` right-thinking ' films


## Run adversarial attacks

In [6]:
!python semi_attack.py -c models/sample-GRU/E9.pytorch --iter 2 --steps 10 --rseed 7 -v \
  --victim_model "distilbert-base-uncased-finetuned-sst-2-english" \
  --victim_sentence "a gorgeous , high-spirited musical from india that exquisitely mixed music , dance , song , and high drama ." \
  --reference_sentence "i finished my final exam at ucla"

2020-12-06 09:42:49.290892: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Model loaded from models/sample-GRU/E9.pytorch

-------Initial Inputs-------
Victim Sentence: a gorgeous , high-spirited musical from india that exquisitely mixed music , dance , song , and high drama . pred:0.9998873472213745
Reference Sentence: i finished my final exam at ucla pred:0.0765596479177475

-------ITERATION 0-------
Best Adv Sentence: i finished my final exam at ucla pred:0.0765596479177475
-------PREDICTIONS-------
1.000 & a gorgeous , high-spirited musical from india that exquisitely mixed music , dance , song , and high drama . \\
0.998 & a confluence , heady jumble of effectively creepy-scary , and dance , and dance , and the whole slo-mo , double-pistoled , ballistic-pyrotechnic hong kong action sequences  \\
0.559 & a smart , steamy mix of sulky teen comedy that plays like a 95 - minute commercial for nba properties .  \\
1

In [12]:
!python semi_attack.py -c models/sample-GRU/E9.pytorch --iter 2 --steps 10 --rseed 42 -v \
  --victim_model "distilbert-base-uncased-finetuned-sst-2-english" \
  --victim_sentence "a strangely compelling and brilliantly acted psychological drama ." \
  --reference_sentence "an absurdist sitcom about alienation , separation and loss ."

2020-12-06 09:48:10.834165: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Model loaded from models/sample-GRU/E9.pytorch

-------Initial Inputs-------
Victim Sentence: a strangely compelling and brilliantly acted psychological drama . pred:0.999883770942688
Reference Sentence: an absurdist sitcom about alienation , separation and loss . pred:0.0025041822809726

-------ITERATION 0-------
Best Adv Sentence: an absurdist sitcom about alienation , separation and loss . pred:0.0025041822809726
-------PREDICTIONS-------
1.000 & a strangely compelling and brilliantly acted psychological drama . \\
1.000 & a chilling tale of a deeply humanistic artist who has been awarded mythic status in contemporary culture  \\
1.000 & a fascinating glimpse of urban life  \\
1.000 & a fascinating glimpse into an urban hades , languid romanticism  \\
0.994 & a fascinating glimpse into the urban heart and the class warfare that embroils tw